 # Time Series Experiment Runner



 This notebook compares multiple forecasting approaches:

 - **Baselines:** Naive, Moving Average

 - **ML Models:** Random Forest, XGBoost

 - **Deep Learning:** LSTM, GRU

 - **Strategies:** Direct vs Recursive forecasting

In [1]:
# === IMPORTS ===
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# Deep Learning imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
tf.get_logger().setLevel('ERROR')  # Suppress TF warnings

# Progress bar (falls back to simple print if not installed)
try:
    from tqdm.notebook import tqdm
    HAS_TQDM = True
except ImportError:
    HAS_TQDM = False
    def tqdm(iterable, desc="", total=None):
        for i, item in enumerate(iterable):
            print(f"  {desc} [{i+1}/{total or '?'}]", end='\r')
            yield item
        print()

print("Imports complete")


2025-11-28 19:44:27.452744: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-28 19:44:27.503993: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 19:44:28.321885: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-28 19:44:31.827120: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different compu

Imports complete


 ## 1. Configuration

In [2]:
# === CONFIGURATION ===
# Modify these to adjust your experiments

TARGET = 'log_price'
HORIZONS = [1, 3, 6, 12]
SEQ_LENGTHS = [4, 6, 12]
TRAIN_END_YEAR = 2019

# Deep learning settings
DL_EPOCHS = 50
DL_BATCH_SIZE = 32
DL_PATIENCE = 10  # Early stopping patience

# Results storage
results = []

print(f"Configuration:")
print(f"  Target: {TARGET}")
print(f"  Horizons: {HORIZONS}")
print(f"  Sequence lengths: {SEQ_LENGTHS}")
print(f"  Train end year: {TRAIN_END_YEAR}")

train_df = pd.read_csv("../tsa_train.csv")
test_df = pd.read_csv("../tsa_test.csv")
train_df['split'] = 'train'
test_df['split'] = 'test'
full_df = pd.concat([train_df, test_df], ignore_index=True)
full_df = full_df.sort_values(['region', 'period_begin']).reset_index(drop=True)

Configuration:
  Target: log_price
  Horizons: [1, 3, 6, 12]
  Sequence lengths: [4, 6, 12]
  Train end year: 2019


 ## 2. Helper Functions

In [3]:
# === HELPER FUNCTIONS ===

def log(msg):
    """Print with timestamp."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def evaluate(y_true, y_pred):
    """Calculate all metrics."""
    return {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred)
    }

def create_features_for_horizon(df, target_col, seq_length, horizon):
    """Create lag/trend features for time series forecasting."""
    feature_dfs = []
    for zip_code in df['region'].unique():
        zip_df = df[df['region'] == zip_code].sort_values('period_begin').copy()
        if len(zip_df) < seq_length + horizon:
            continue
        
        # Lag features
        for lag in range(1, seq_length + 1):
            shift_amount = lag + horizon - 1
            zip_df[f'lag_{lag}'] = zip_df[target_col].shift(shift_amount)
        
        # Trend features (log-spaced)
        n_trends = min(int(np.ceil(np.log2(seq_length))), 5)
        trend_points = np.unique(np.geomspace(2, seq_length, n_trends, dtype=int))
        for n in trend_points:
            if n <= seq_length:
                zip_df[f'trend_{n}'] = zip_df['lag_1'] - zip_df[f'lag_{n}']
        
        # Derived features
        if seq_length >= 3:
            zip_df['momentum'] = zip_df['lag_1'] - 2*zip_df['lag_2'] + zip_df['lag_3']
        else:
            zip_df['momentum'] = zip_df['lag_1'] - zip_df['lag_2']
        zip_df['volatility'] = zip_df[target_col].rolling(window=seq_length).std().shift(horizon)
        zip_df['rolling_mean'] = zip_df[target_col].rolling(window=seq_length).mean().shift(horizon)
        
        zip_df['target'] = zip_df[target_col]
        feature_dfs.append(zip_df)
    
    if not feature_dfs:
        return None
    return pd.concat(feature_dfs, ignore_index=True).dropna().reset_index(drop=True)

def get_feature_columns(df):
    """Return only engineered feature columns."""
    lag_cols = sorted([c for c in df.columns if c.startswith('lag_')], 
                      key=lambda x: int(x.split('_')[1]))
    trend_cols = sorted([c for c in df.columns if c.startswith('trend_')],
                        key=lambda x: int(x.split('_')[1]))
    derived_cols = [c for c in ['momentum', 'volatility', 'rolling_mean'] if c in df.columns]
    return lag_cols + trend_cols + derived_cols

def get_lag_columns(df, seq_length):
    """Return only lag columns (for LSTM sequence input)."""
    return [f'lag_{i}' for i in range(1, seq_length + 1)]

def train_test_split_temporal(df, train_end_year):
    """Split by year (temporal)."""
    train = df[df['year'] <= train_end_year].copy()
    test = df[df['year'] > train_end_year].copy()
    return train, test

print("✅ Helper functions defined")


✅ Helper functions defined


 ## 3. Model Definitions

In [4]:
# === ML MODELS ===

def get_ml_models():
    """Return dict of ML model_name -> model_instance."""
    return {
        'RF_default': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        'RF_tuned': RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1),
        'XGB_default': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
        'XGB_tuned': XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42, verbosity=0),
    }

# === DEEP LEARNING MODELS ===

def build_lstm_small(seq_length, n_features=1):
    """LSTM: 32 units, 1 layer."""
    model = Sequential([
        LSTM(32, input_shape=(seq_length, n_features)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_lstm_medium(seq_length, n_features=1):
    """LSTM: 64 units, 2 layers."""
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(seq_length, n_features)),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_lstm_large(seq_length, n_features=1):
    """LSTM: 128 units, 2 layers + dropout."""
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(seq_length, n_features)),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_gru_small(seq_length, n_features=1):
    """GRU: 32 units, 1 layer."""
    model = Sequential([
        GRU(32, input_shape=(seq_length, n_features)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def build_gru_medium(seq_length, n_features=1):
    """GRU: 64 units, 2 layers."""
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=(seq_length, n_features)),
        GRU(32),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def get_dl_models():
    """Return dict of DL model builders."""
    return {
        'LSTM_small': build_lstm_small,
        'LSTM_medium': build_lstm_medium,
        'LSTM_large': build_lstm_large,
        'GRU_small': build_gru_small,
        'GRU_medium': build_gru_medium,
    }

print("✅ Model definitions complete")
print(f"   ML models: {list(get_ml_models().keys())}")
print(f"   DL models: {list(get_dl_models().keys())}")


✅ Model definitions complete
   ML models: ['RF_default', 'RF_tuned', 'XGB_default', 'XGB_tuned']
   DL models: ['LSTM_small', 'LSTM_medium', 'LSTM_large', 'GRU_small', 'GRU_medium']


 ## 4. Baseline Models

In [5]:
# === BASELINE MODELS ===

def naive_forecast(test_df):
    """Naive: predict lag_1 (most recent known value)."""
    return test_df['lag_1'].values

def moving_avg_forecast(test_df, seq_length):
    """Moving average of all lags."""
    lag_cols = [f'lag_{i}' for i in range(1, seq_length + 1)]
    return test_df[lag_cols].mean(axis=1).values

print("✅ Baseline models defined")


✅ Baseline models defined


 ## 5. Recursive Forecasting

In [6]:
# === RECURSIVE FORECASTING ===

def recursive_forecast_ml(model, test_df, feature_cols, seq_length, target_horizon):
    """Use horizon=1 ML model to recursively predict multiple steps."""
    predictions = []
    
    for idx in range(len(test_df)):
        row = test_df.iloc[idx]
        sequence = [row[f'lag_{i}'] for i in range(1, seq_length + 1)]
        
        for step in range(target_horizon):
            features = sequence[:seq_length]
            
            # Add trend features
            n_trends = min(int(np.ceil(np.log2(seq_length))), 5)
            trend_points = np.unique(np.geomspace(2, seq_length, n_trends, dtype=int))
            for n in trend_points:
                if n <= seq_length:
                    features.append(features[0] - features[n-1])
            
            # Add derived features
            if seq_length >= 3:
                features.append(features[0] - 2*features[1] + features[2])
            else:
                features.append(features[0] - features[1])
            features.append(np.std(sequence[:seq_length]))
            features.append(np.mean(sequence[:seq_length]))
            
            pred = model.predict(np.array(features).reshape(1, -1))[0]
            sequence = [pred] + sequence[:-1]
        
        predictions.append(pred)
    
    return np.array(predictions)

def recursive_forecast_dl(model, test_sequences, seq_length, target_horizon, scaler=None):
    """Use horizon=1 DL model to recursively predict multiple steps."""
    predictions = []
    
    for seq in test_sequences:
        current_seq = seq.copy()
        
        for step in range(target_horizon):
            # Reshape for prediction
            X = current_seq.reshape(1, seq_length, 1)
            pred = model.predict(X, verbose=0)[0, 0]
            
            # Slide window
            current_seq = np.roll(current_seq, -1)
            current_seq[-1] = pred
        
        predictions.append(pred)
    
    return np.array(predictions)

print("✅ Recursive forecasting functions defined")


✅ Recursive forecasting functions defined


 ## 6. Main Experiment Runner

In [7]:
# === EXPERIMENT RUNNER ===

def run_all_experiments(full_df, run_baselines=True, run_ml=True, run_dl=True, run_recursive=True):
    """
    Run all experiments.
    
    Parameters:
    -----------
    full_df : DataFrame with preprocessed data
    run_baselines : bool, run Naive/MovingAvg
    run_ml : bool, run RF/XGBoost
    run_dl : bool, run LSTM/GRU
    run_recursive : bool, run recursive forecasting comparison
    """
    global results
    results = []
    
    # Count total experiments
    n_configs = len(HORIZONS) * len(SEQ_LENGTHS)
    n_baselines = 2 if run_baselines else 0
    n_ml = len(get_ml_models()) if run_ml else 0
    n_dl = len(get_dl_models()) if run_dl else 0
    total = n_configs * (n_baselines + n_ml + n_dl)
    
    log(f"Starting {total} experiments")
    print("=" * 60)
    
    exp_count = 0
    
    # ========================================
    # DIRECT FORECASTING
    # ========================================
    for horizon in HORIZONS:
        for seq_length in SEQ_LENGTHS:
            print(f"\n{'='*60}")
            log(f"HORIZON={horizon}, SEQ_LENGTH={seq_length}")
            print("=" * 60)
            
            # Create features
            df_features = create_features_for_horizon(full_df, TARGET, seq_length, horizon)
            if df_features is None:
                log("⚠️  SKIP: Not enough data")
                continue
            
            # Split data
            train_df, test_df = train_test_split_temporal(df_features, TRAIN_END_YEAR)
            feature_cols = get_feature_columns(train_df)
            lag_cols = get_lag_columns(train_df, seq_length)
            
            X_train = train_df[feature_cols].values
            y_train = train_df['target'].values
            X_test = test_df[feature_cols].values
            y_test = test_df['target'].values
            
            # For DL: extract just lag sequences
            X_train_seq = train_df[lag_cols].values.reshape(-1, seq_length, 1)
            X_test_seq = test_df[lag_cols].values.reshape(-1, seq_length, 1)
            
            # Scale for DL
            scaler = StandardScaler()
            X_train_seq_scaled = scaler.fit_transform(X_train_seq.reshape(-1, seq_length)).reshape(-1, seq_length, 1)
            X_test_seq_scaled = scaler.transform(X_test_seq.reshape(-1, seq_length)).reshape(-1, seq_length, 1)
            
            log(f"Train: {len(train_df):,} | Test: {len(test_df):,} | Features: {len(feature_cols)}")
            
            # --- BASELINES ---
            if run_baselines:
                for name, forecast_fn in [('Naive', naive_forecast), ('MovingAvg', lambda df: moving_avg_forecast(df, seq_length))]:
                    exp_count += 1
                    y_pred = forecast_fn(test_df)
                    metrics = evaluate(y_test, y_pred)
                    results.append({
                        'strategy': 'direct', 'model': name, 'horizon': horizon,
                        'seq_length': seq_length, **metrics, 'n_train': len(train_df), 'n_test': len(test_df)
                    })
                    print(f"  ✓ [{exp_count}/{total}] {name:<15} RMSE={metrics['rmse']:.4f}  R²={metrics['r2']:.4f}")
            
            # --- ML MODELS ---
            if run_ml:
                for name, model in get_ml_models().items():
                    exp_count += 1
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)
                    metrics = evaluate(y_test, y_pred)
                    results.append({
                        'strategy': 'direct', 'model': name, 'horizon': horizon,
                        'seq_length': seq_length, **metrics, 'n_train': len(train_df), 'n_test': len(test_df)
                    })
                    print(f"  ✓ [{exp_count}/{total}] {name:<15} RMSE={metrics['rmse']:.4f}  R²={metrics['r2']:.4f}")
            
            # --- DEEP LEARNING ---
            if run_dl:
                early_stop = EarlyStopping(monitor='val_loss', patience=DL_PATIENCE, restore_best_weights=True, verbose=0)
                
                for name, build_fn in get_dl_models().items():
                    exp_count += 1
                    print(f"  ⏳ [{exp_count}/{total}] {name:<15} training...", end='\r')
                    
                    model = build_fn(seq_length, n_features=1)
                    model.fit(
                        X_train_seq_scaled, y_train,
                        validation_split=0.2,
                        epochs=DL_EPOCHS,
                        batch_size=DL_BATCH_SIZE,
                        callbacks=[early_stop],
                        verbose=0
                    )
                    y_pred = model.predict(X_test_seq_scaled, verbose=0).flatten()
                    metrics = evaluate(y_test, y_pred)
                    results.append({
                        'strategy': 'direct', 'model': name, 'horizon': horizon,
                        'seq_length': seq_length, **metrics, 'n_train': len(train_df), 'n_test': len(test_df)
                    })
                    print(f"  ✓ [{exp_count}/{total}] {name:<15} RMSE={metrics['rmse']:.4f}  R²={metrics['r2']:.4f}")
                    
                    # Clear memory
                    tf.keras.backend.clear_session()
    
    # ========================================
    # RECURSIVE FORECASTING
    # ========================================
    if run_recursive:
        print(f"\n{'='*60}")
        log("RECURSIVE FORECASTING")
        print("=" * 60)
        
        # Find best horizon=1 ML model
        h1_ml = [r for r in results if r['horizon'] == 1 and r['model'] in get_ml_models().keys()]
        h1_dl = [r for r in results if r['horizon'] == 1 and r['model'] in get_dl_models().keys()]
        
        if h1_ml:
            best_ml = min(h1_ml, key=lambda x: x['rmse'])
            log(f"Best ML (h=1): {best_ml['model']} (RMSE={best_ml['rmse']:.4f})")
        if h1_dl:
            best_dl = min(h1_dl, key=lambda x: x['rmse'])
            log(f"Best DL (h=1): {best_dl['model']} (RMSE={best_dl['rmse']:.4f})")
        
        for seq_length in SEQ_LENGTHS:
            # Prepare horizon=1 data
            df_h1 = create_features_for_horizon(full_df, TARGET, seq_length, horizon=1)
            if df_h1 is None:
                continue
            
            train_df, test_df = train_test_split_temporal(df_h1, TRAIN_END_YEAR)
            feature_cols = get_feature_columns(train_df)
            lag_cols = get_lag_columns(train_df, seq_length)
            
            X_train = train_df[feature_cols].values
            y_train = train_df['target'].values
            
            # Train best ML model
            if h1_ml:
                best_ml_model = get_ml_models()[best_ml['model']]
                best_ml_model.fit(X_train, y_train)
            
            # Train best DL model
            if h1_dl:
                X_train_seq = train_df[lag_cols].values.reshape(-1, seq_length, 1)
                scaler = StandardScaler()
                X_train_seq_scaled = scaler.fit_transform(X_train_seq.reshape(-1, seq_length)).reshape(-1, seq_length, 1)
                
                best_dl_model = get_dl_models()[best_dl['model']](seq_length, n_features=1)
                early_stop = EarlyStopping(monitor='val_loss', patience=DL_PATIENCE, restore_best_weights=True, verbose=0)
                best_dl_model.fit(X_train_seq_scaled, y_train, validation_split=0.2, 
                                  epochs=DL_EPOCHS, batch_size=DL_BATCH_SIZE, callbacks=[early_stop], verbose=0)
            
            # Test at each target horizon
            for target_horizon in [3, 6, 12]:
                df_direct = create_features_for_horizon(full_df, TARGET, seq_length, target_horizon)
                if df_direct is None:
                    continue
                
                _, test_direct = train_test_split_temporal(df_direct, TRAIN_END_YEAR)
                y_test = test_direct['target'].values
                n_test = min(len(test_df), len(test_direct))
                
                log(f"seq={seq_length}, target_horizon={target_horizon}")
                
                # Recursive ML
                if h1_ml:
                    y_pred = recursive_forecast_ml(best_ml_model, test_df.head(n_test), feature_cols, seq_length, target_horizon)
                    metrics = evaluate(y_test[:len(y_pred)], y_pred)
                    results.append({
                        'strategy': 'recursive', 'model': f"{best_ml['model']}_recursive", 
                        'horizon': target_horizon, 'seq_length': seq_length, 
                        **metrics, 'n_train': len(train_df), 'n_test': len(y_pred)
                    })
                    print(f"  ✓ {best_ml['model']}_recursive: RMSE={metrics['rmse']:.4f}  R²={metrics['r2']:.4f}")
                
                # Recursive DL
                if h1_dl:
                    test_seq = test_df.head(n_test)[lag_cols].values
                    test_seq_scaled = scaler.transform(test_seq)
                    y_pred = recursive_forecast_dl(best_dl_model, test_seq_scaled, seq_length, target_horizon)
                    metrics = evaluate(y_test[:len(y_pred)], y_pred)
                    results.append({
                        'strategy': 'recursive', 'model': f"{best_dl['model']}_recursive",
                        'horizon': target_horizon, 'seq_length': seq_length,
                        **metrics, 'n_train': len(train_df), 'n_test': len(y_pred)
                    })
                    print(f"  ✓ {best_dl['model']}_recursive: RMSE={metrics['rmse']:.4f}  R²={metrics['r2']:.4f}")
                
                tf.keras.backend.clear_session()
    
    print(f"\n{'='*60}")
    log(f"✅ COMPLETE! {len(results)} experiments recorded.")
    print("=" * 60)
    
    return pd.DataFrame(results)

print("✅ Experiment runner ready")


✅ Experiment runner ready


 ## 7. Run Experiments



 Adjust flags to run subsets:

 - `run_baselines=True` - Naive, Moving Average

 - `run_ml=True` - Random Forest, XGBoost

 - `run_dl=True` - LSTM, GRU (slower)

 - `run_recursive=True` - Compare direct vs recursive

In [ ]:
# === RUN EXPERIMENTS ===

# Refern to experiments_results.csv for saved results
# As this is a VERY long running cell (30-60 minutes on a decent GPU)
# Conisider only running with a subset of options for quick testing

# Run all experiments (this will take a while with DL models)
results_df = run_all_experiments(
    full_df,
    run_baselines=True,
    run_ml=True,
    run_dl=True,       # Set False for quick test
    run_recursive=True  # Set False for quick test
)


[19:44:33] Starting 132 experiments

[19:44:33] HORIZON=1, SEQ_LENGTH=4
[19:44:33] Train: 8,638 | Test: 2,443 | Features: 9
  ✓ [1/132] Naive           RMSE=0.1694  R²=0.8330
  ✓ [2/132] MovingAvg       RMSE=0.1856  R²=0.7995
  ✓ [3/132] RF_default      RMSE=0.1734  R²=0.8250
  ✓ [4/132] RF_tuned        RMSE=0.1711  R²=0.8295
  ✓ [5/132] XGB_default     RMSE=0.1877  R²=0.7949
  ✓ [6/132] XGB_tuned       RMSE=0.1788  R²=0.8140


2025-11-28 19:44:38.955088: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


  ✓ [7/132] LSTM_small      RMSE=0.1683  R²=0.8352
  ✓ [8/132] LSTM_medium     RMSE=0.7293  R²=-2.0963
  ✓ [9/132] LSTM_large      RMSE=0.3035  R²=0.4639
  ✓ [10/132] GRU_small       RMSE=0.6052  R²=-1.1323
  ✓ [11/132] GRU_medium      RMSE=1.1807  R²=-7.1150

[19:46:12] HORIZON=1, SEQ_LENGTH=6
[19:46:12] Train: 8,412 | Test: 2,443 | Features: 12
  ✓ [12/132] Naive           RMSE=0.1694  R²=0.8330
  ✓ [13/132] MovingAvg       RMSE=0.1925  R²=0.7844
  ✓ [14/132] RF_default      RMSE=0.1738  R²=0.8242
  ✓ [15/132] RF_tuned        RMSE=0.1726  R²=0.8265
  ✓ [16/132] XGB_default     RMSE=0.1860  R²=0.7987
  ✓ [17/132] XGB_tuned       RMSE=0.1775  R²=0.8167
  ✓ [18/132] LSTM_small      RMSE=0.1655  R²=0.8406
  ✓ [19/132] LSTM_medium     RMSE=0.7229  R²=-2.0415
  ✓ [20/132] LSTM_large      RMSE=0.3231  R²=0.3922
  ✓ [21/132] GRU_small       RMSE=0.7397  R²=-2.1846
  ✓ [22/132] GRU_medium      RMSE=0.6391  R²=-1.3776

[19:48:12] HORIZON=1, SEQ_LENGTH=12
[19:48:13] Train: 7,734 | Test: 2,443 |

KeyboardInterrupt: 

 ## 8. Analyze Results

In [ ]:
# === VIEW RESULTS ===

print("\n📊 ALL RESULTS")
print("=" * 80)
display(results_df.sort_values(['horizon', 'rmse']))


In [ ]:
# === BEST MODEL BY HORIZON ===

print("\n🏆 BEST MODEL BY HORIZON")
print("=" * 60)

for horizon in HORIZONS:
    subset = results_df[results_df['horizon'] == horizon]
    if len(subset) == 0:
        continue
    best = subset.loc[subset['rmse'].idxmin()]
    print(f"\nHorizon {horizon}:")
    print(f"  Best: {best['model']} ({best['strategy']}, seq={best['seq_length']})")
    print(f"  RMSE: {best['rmse']:.4f} | MAE: {best['mae']:.4f} | R²: {best['r2']:.4f}")


In [ ]:
# === DIRECT VS RECURSIVE COMPARISON ===

print("\n⚔️  DIRECT vs RECURSIVE COMPARISON")
print("=" * 60)

for horizon in [3, 6, 12]:
    direct = results_df[(results_df['horizon'] == horizon) & (results_df['strategy'] == 'direct')]
    recursive = results_df[(results_df['horizon'] == horizon) & (results_df['strategy'] == 'recursive')]
    
    if len(direct) == 0 or len(recursive) == 0:
        continue
    
    best_direct = direct.loc[direct['rmse'].idxmin()]
    best_recursive = recursive.loc[recursive['rmse'].idxmin()]
    
    print(f"\nHorizon {horizon}:")
    print(f"  Direct:    {best_direct['model']:<25} RMSE={best_direct['rmse']:.4f}")
    print(f"  Recursive: {best_recursive['model']:<25} RMSE={best_recursive['rmse']:.4f}")
    
    diff = (best_recursive['rmse'] - best_direct['rmse']) / best_direct['rmse'] * 100
    winner = "✅ Direct" if diff > 0 else "✅ Recursive"
    print(f"  {winner} wins by {abs(diff):.1f}%")


In [ ]:
# === ML VS DL COMPARISON ===

print("\n🤖 ML vs DEEP LEARNING COMPARISON")
print("=" * 60)

ml_models = list(get_ml_models().keys())
dl_models = list(get_dl_models().keys())

for horizon in HORIZONS:
    direct = results_df[(results_df['horizon'] == horizon) & (results_df['strategy'] == 'direct')]
    
    ml_results = direct[direct['model'].isin(ml_models)]
    dl_results = direct[direct['model'].isin(dl_models)]
    
    if len(ml_results) == 0 or len(dl_results) == 0:
        continue
    
    best_ml = ml_results.loc[ml_results['rmse'].idxmin()]
    best_dl = dl_results.loc[dl_results['rmse'].idxmin()]
    
    print(f"\nHorizon {horizon}:")
    print(f"  Best ML: {best_ml['model']:<15} RMSE={best_ml['rmse']:.4f}")
    print(f"  Best DL: {best_dl['model']:<15} RMSE={best_dl['rmse']:.4f}")
    
    winner = "ML" if best_ml['rmse'] < best_dl['rmse'] else "DL"
    diff = abs(best_ml['rmse'] - best_dl['rmse']) / min(best_ml['rmse'], best_dl['rmse']) * 100
    print(f"  ✅ {winner} wins by {diff:.1f}%")


 ## 9. Save Results

In [ ]:
# === SAVE ===

results_df.to_csv('experiment_results.csv', index=False)
print("✅ Results saved to experiment_results.csv")

# Also save a summary
summary = results_df.groupby(['horizon', 'strategy']).apply(
    lambda x: x.loc[x['rmse'].idxmin()][['model', 'seq_length', 'rmse', 'r2']]
).reset_index()
summary.to_csv('experiment_summary.csv', index=False)
print("✅ Summary saved to experiment_summary.csv")

display(summary)